<a href="https://colab.research.google.com/github/WenyuCho/Generative-AI/blob/main/Homework/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# preprocessing

載入資料集(QMSum)

In [5]:
# === 建立資料夾並 Clone QMSum GitHub ===
!git clone https://github.com/Yale-LILY/QMSum.git

Cloning into 'QMSum'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 809 (delta 1), reused 0 (delta 0), pack-reused 804 (from 1)
Receiving objects: 100% (809/809), 13.76 MiB | 21.07 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [6]:
pwd

'/content'

In [7]:
!mkdir RAGLLM
!mkdir RAGLLM/data

mkdir: cannot create directory ‘RAGLLM’: File exists
mkdir: cannot create directory ‘RAGLLM/data’: File exists


將每一行為一筆資料的 .jsonl 格式（如 test set）轉換為標準 .json 檔，並加上 relevant 編號。

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
JSONL to JSON Converter - 固定處理 QMSum 測試資料
"""

import json
from pathlib import Path

INPUT_FILE = "QMSum/data/ALL/jsonl/test.jsonl"
OUTPUT_FILE = "QMSum/data/ALL/jsonl/converted_test.json"

def convert_jsonl_to_json(input_file, output_file):
    result = {}

    print(f"正在讀取文件: {input_file}")
    with open(input_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                meeting_data = json.loads(line)
                meeting_id = f"m_test_{line_num - 1}"

                if 'meeting_transcripts' in meeting_data and meeting_data['meeting_transcripts']:
                    for i, transcript in enumerate(meeting_data['meeting_transcripts']):
                        transcript['relevant'] = str(i)

                result[meeting_id] = meeting_data
            except json.JSONDecodeError as e:
                print(f"錯誤: 第{line_num}行JSON格式錯誤 - {e}")
                continue

    # 儲存 JSON
    print(f"正在寫入文件: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)

    # 統計
    print(f"\n轉換完成!")
    print(f"會議數: {len(result)}")
    print(f"輸出文件: {output_file}")
    print(f"檔案大小: {Path(output_file).stat().st_size / 1024:.1f} KB")

    # 顯示前 5 筆統計
    print(f"\n各會議摘要:")
    for meeting_id, meeting_data in list(result.items())[:5]:
        print(f"   {meeting_id}:")
        print(f"     - 主題數: {len(meeting_data.get('topic_list', []))}")
        print(f"     - 一般查詢: {len(meeting_data.get('general_query_list', []))}")
        print(f"     - 特定查詢: {len(meeting_data.get('specific_query_list', []))}")
        print(f"     - 逐字稿段落: {len(meeting_data.get('meeting_transcripts', []))}")
    if len(result) > 5:
        print(f"   ... 還有 {len(result) - 5} 個會議")

    return result

if __name__ == '__main__':
    convert_jsonl_to_json(INPUT_FILE, OUTPUT_FILE)


正在讀取文件: QMSum/data/ALL/jsonl/test.jsonl
正在寫入文件: QMSum/data/ALL/jsonl/converted_test.json

轉換完成!
會議數: 35
輸出文件: QMSum/data/ALL/jsonl/converted_test.json
檔案大小: 3871.0 KB

各會議摘要:
   m_test_0:
     - 主題數: 6
     - 一般查詢: 1
     - 特定查詢: 12
     - 逐字稿段落: 133
   m_test_1:
     - 主題數: 4
     - 一般查詢: 1
     - 特定查詢: 6
     - 逐字稿段落: 668
   m_test_2:
     - 主題數: 2
     - 一般查詢: 1
     - 特定查詢: 4
     - 逐字稿段落: 376
   m_test_3:
     - 主題數: 5
     - 一般查詢: 1
     - 特定查詢: 6
     - 逐字稿段落: 804
   m_test_4:
     - 主題數: 4
     - 一般查詢: 1
     - 特定查詢: 4
     - 逐字稿段落: 726
   ... 還有 30 個會議


將 JSON 格式的 QMSum 會議資料，轉換為 segments.csv 和 queries.csv，供後續 embedding 和問答流程使用。

In [9]:
import json
import csv
import re
from pathlib import Path

INPUT_JSON_PATH = "QMSum/data/ALL/jsonl/converted_test.json"
SEGMENTS_CSV_PATH = "RAGLLM/data/segments.csv"
QUERIES_CSV_PATH = "RAGLLM/data/queries.csv"

def convert_json_to_csv(input_json_path, segments_csv_path, queries_csv_path):
    segment_rows = []
    query_rows = []
    brace_pattern = re.compile(r'\{[^}]+\}')  # 移除轉義標記

    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for meeting_id, meeting_data in data.items():
        transcripts = meeting_data.get("meeting_transcripts", [])
        segment_count = len(transcripts)

        # 建立逐字稿片段
        for t in transcripts:
            content = t.get("content", "")
            content_cleaned = brace_pattern.sub("", content).strip()
            segment_rows.append({
                "meeting_id": meeting_id,
                "segment_count": segment_count,
                "segment_id": t.get("relevant", ""),
                "speaker": t.get("speaker", ""),
                "content": content_cleaned
            })

        # 一般查詢
        for q in meeting_data.get("general_query_list", []):
            query_rows.append({
                "meeting_id": meeting_id,
                "query_type": "general",
                "query": q.get("query", ""),
                "answer": q.get("answer", ""),
                "relevant_text_span": ""
            })

        # 特定查詢
        for q in meeting_data.get("specific_query_list", []):
            query_rows.append({
                "meeting_id": meeting_id,
                "query_type": "specific",
                "query": q.get("query", ""),
                "answer": q.get("answer", ""),
                "relevant_text_span": q.get("relevant_text_span", "")
            })

    # 輸出 segments.csv（加雙引號）
    with open(segments_csv_path, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["meeting_id", "segment_count", "segment_id", "speaker", "content"], quoting=csv.QUOTE_ALL)
        writer.writeheader()
        writer.writerows(segment_rows)

    # 輸出 queries.csv
    with open(queries_csv_path, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["meeting_id", "query_type", "query", "answer", "relevant_text_span"])
        writer.writeheader()
        writer.writerows(query_rows)

    print("JSON 轉 CSV 完成：")
    print(f"   segments.csv：{len(segment_rows)} 行 → {segments_csv_path}")
    print(f"   queries.csv：{len(query_rows)} 行 → {queries_csv_path}")

# 直接執行
if __name__ == "__main__":
    convert_json_to_csv(INPUT_JSON_PATH, SEGMENTS_CSV_PATH, QUERIES_CSV_PATH)


JSON 轉 CSV 完成：
   segments.csv：20718 行 → RAGLLM/data/segments.csv
   queries.csv：281 行 → RAGLLM/data/queries.csv


將 segments.csv 每個段落轉為向量嵌入，並以字典格式儲存為 segments_bge_base_en.pt，供後續問答系統使用。

In [10]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import time
from pathlib import Path

CSV_PATH = "RAGLLM/data/segments.csv"
OUTPUT_PT_PATH = "RAGLLM/data/segments_bge_base_en.pt"
EMBEDDING_MODEL = "BAAI/bge-base-en-v1.5"
BATCH_SIZE = 64

def convert_csv_to_pt():
    print(f"載入 embedding 模型：{EMBEDDING_MODEL}")
    model = SentenceTransformer(EMBEDDING_MODEL)

    start_time = time.time()
    df = pd.read_csv(CSV_PATH)

    required_columns = ['meeting_id', 'segment_count', 'segment_id', 'speaker', 'content']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"缺少必要欄位: {col}")

    print(f"原始段落數：{len(df)}")
    df["content"] = df["content"].fillna("")
    df = df[df["content"].str.strip().astype(bool)]
    print(f"過濾後有效段落：{len(df)}")

    data = {}
    total = len(df)
    for i in range(0, total, BATCH_SIZE):
        batch = df.iloc[i:i+BATCH_SIZE].copy()
        contents = batch["content"].tolist()
        embeddings = model.encode(contents, convert_to_tensor=True, show_progress_bar=False)

        for j, (_, row) in enumerate(batch.iterrows()):
            key = f"{row['meeting_id']}_segment_{row['segment_id']}"
            data[key] = {
                "meeting_id": row["meeting_id"],
                "segment_id": str(row["segment_id"]),
                "speaker": row["speaker"],
                "content": row["content"],
                "segment_count": int(row["segment_count"]),
                "embedding": embeddings[j].cpu()
            }

    # 儲存 .pt
    torch.save(data, OUTPUT_PT_PATH)
    duration = time.time() - start_time

    print(f"\n儲存完成！共處理 {len(data)} 段落，耗時 {duration:.2f} 秒")
    print(f"輸出檔案：{OUTPUT_PT_PATH}")

    # 範例輸出
    sample_key = next(iter(data))
    sample = data[sample_key]
    print(f"\n範例段落：")
    print(f" Key: {sample_key}")
    print(f" Content: {sample['content'][:50]}...")
    print(f" Embedding shape: {sample['embedding'].shape}")

if __name__ == "__main__":
    convert_csv_to_pt()


載入 embedding 模型：BAAI/bge-base-en-v1.5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

原始段落數：20718
過濾後有效段落：19862

儲存完成！共處理 19862 段落，耗時 72.04 秒
輸出檔案：RAGLLM/data/segments_bge_base_en.pt

範例段落：
 Key: m_test_0_segment_0
 Content: Good afternoon, everyone. Welcome to the Children,...
 Embedding shape: torch.Size([768])


# funcion

1. 安裝與套件載入區

In [2]:
# 安裝必要套件（如未安裝）
!pip install -q sentence-transformers transformers accelerate
!pip uninstall -y bitsandbytes
!pip install bitsandbytes --prefer-binary --upgrade --no-cache-dir

# 載入套件
import json
import os
import torch
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import re

from sentence_transformers import SentenceTransformer
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
)
from transformers.utils import logging as transformers_logging

transformers_logging.set_verbosity_error()


Found existing installation: bitsandbytes 0.46.0
Uninstalling bitsandbytes-0.46.0:
  Successfully uninstalled bitsandbytes-0.46.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 90.8 MB/s eta 0:00:00


In [3]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes==0.46.0 --prefer-binary --no-cache-dir

Found existing installation: bitsandbytes 0.46.0
Uninstalling bitsandbytes-0.46.0:
  Successfully uninstalled bitsandbytes-0.46.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 90.6 MB/s eta 0:00:00


2. 參數與模型初始化

❗❗❗注意：如果使用特定 huggingface 上，需要 API Tokens 的 model，要在Secret內帶入Access Tokens❗❗❗

In [4]:
from huggingface_hub import login
from google.colab import userdata

# 從 Colab Secrets 中取得 token
hf_token = userdata.get("HUGGINGFACE_TOKEN")
login(token=hf_token)

# === 基本參數設定 ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOP_K = 10
MAX_TOKENS = 7500

EMBED_MODEL = "BAAI/bge-base-en-v1.5"
LLM_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"

# === 載入 Embedding 模型 ===
print("\n[Embedding] 載入 bge-base-en-v1.5...")
embedder = SentenceTransformer(EMBED_MODEL, device=DEVICE)

# === 載入 LLM 模型（4bit 量化）===
print("\n[LLM] 載入 LLaMA-3 模型（4bit）...")
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, use_fast=True)
tokenizer.use_default_system_prompt = False

model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL, device_map="auto", quantization_config=quant_config
)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)



[Embedding] 載入 bge-base-en-v1.5...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



[LLM] 載入 LLaMA-3 模型（4bit）...


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

3. 基礎工具函式

In [5]:
def truncate_by_tokens(text, tokenizer, max_tokens=MAX_TOKENS):
    input_ids = tokenizer.encode(text, truncation=True, max_length=max_tokens, return_tensors=None)
    return tokenizer.decode(input_ids, skip_special_tokens=True)

def get_embedding(text):
    return embedder.encode(text, convert_to_tensor=True).to(DEVICE)

def clean_generated_output(text):
    text = text.strip()
    for prefix in ["Answer:", "A:", "答：", "Response:"]:
        if text.lower().startswith(prefix.lower()):
            text = text[len(prefix):].strip()
    return text

def is_valid_segment(s):
    return (
        isinstance(s.get("content", ""), str) and s["content"].strip() and
        isinstance(s.get("speaker", ""), str)
    )


4. 模型問答函式（含 fallback）

In [6]:
def ask_llm(context, query):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant. Answer the user's query as specifically and factually as possible "
                "based only on the transcript excerpts. Do not guess or invent information. "
                "If the query is too vague or irrelevant, reply: <<UNANSWERABLE>>."
            )
        },
        {
            "role": "user",
            "content": f"Transcript:\n{context}\n\nQuery:\n{query}"
        }
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    raw = llm(prompt, return_full_text=False)[0]['generated_text']
    cleaned = clean_generated_output(raw).strip()

    if (
        cleaned.upper() == "<<UNANSWERABLE>>" or
        len(cleaned) < 3 or
        cleaned.lower() in {"barry", "barry hugh"} or
        not re.match(r"^[A-Z].{3,}[.!?]?$", cleaned)
    ):
        return "<<UNANSWERABLE>>"
    return cleaned

def ask_llm_fallback(full_context, query):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant tasked with summarizing full transcripts to answer user queries. "
                "Summarize relevant parts clearly and concisely. Use only the information from the transcript."
            )
        },
        {
            "role": "user",
            "content": f"Transcript:\n{full_context}\n\nQuery:\n{query}"
        }
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    raw = llm(prompt, return_full_text=False)[0]['generated_text']
    return clean_generated_output(raw).strip()


5. SelfRoute 主推理函式

In [7]:
def selfroute_inference(query, reference, full_context, global_segments):
    query_emb = get_embedding(f"{query}\n\n{full_context}").to(DEVICE)
    query_emb = F.normalize(query_emb, p=2, dim=0)

    filtered_segments = [s for s in global_segments if is_valid_segment(s)]
    if not filtered_segments:
        return {
            "query": query,
            "reference": reference,
            "used_fallback": True,
            "answer": "No valid segments found.",
            "top_segments": []
        }

    all_embeddings = torch.stack([s["embedding"].to(DEVICE) for s in filtered_segments])
    all_embeddings = F.normalize(all_embeddings, p=2, dim=1)
    sim_scores = torch.matmul(all_embeddings, query_emb)
    top_k_values, top_k_indices = torch.topk(sim_scores, k=min(TOP_K, len(filtered_segments)))

    top_segments = []
    for score, idx in zip(top_k_values, top_k_indices):
        s = filtered_segments[idx]
        top_segments.append({
            "id": idx.item(),
            "speaker": s["speaker"],
            "text": s["content"],
            "similarity": round(score.item(), 4)
        })

    retrieved_context = "\n\n".join([f"{s['speaker']}: {s['text']}" for s in top_segments if s["text"].strip()])
    rag_answer = ask_llm(retrieved_context, query)

    if rag_answer.strip() == "<<UNANSWERABLE>>":
        fallback_context = truncate_by_tokens(retrieved_context, tokenizer, max_tokens=MAX_TOKENS)
        print(f"\n[Fallback] 使用 fallback 策略，因為 RAG 回答無效：{rag_answer}")
        final_answer = ask_llm_fallback(fallback_context, query)
        used_fallback = True
    else:
        final_answer = rag_answer
        used_fallback = False

    return {
        "query": query,
        "reference": reference,
        "used_fallback": used_fallback,
        "answer": final_answer,
        "top_segments": top_segments
    }


6. 主程式（使用測試資料執行）

In [8]:
queries_csv_path = "RAGLLM/data/queries.csv"
segments_csv_path = "RAGLLM/data/segments.csv"
segments_pt_path = "RAGLLM/data/segments_bge_base_en.pt"
output_json_path = "output/retrieval_topk.json"

# 載入資料
queries_df = pd.read_csv(queries_csv_path)
segments_df = pd.read_csv(segments_csv_path)
segment_data = torch.load(segments_pt_path, map_location="cpu")
global_segments = [v for k, v in segment_data.items() if is_valid_segment(v)]

# 問答流程
results = []
for _, row in tqdm(queries_df.iterrows(), total=len(queries_df), desc="自動問答中"):
    meeting_id = row["meeting_id"]
    query = row["query"]
    reference = row.get("answer", "")
    matched_contents = segments_df[segments_df["meeting_id"] == meeting_id]["content"].dropna().tolist()
    full_context = " ".join(matched_contents).strip()
    result = selfroute_inference(query, reference, full_context, global_segments)
    results.append(result)

# 輸出 JSON
os.makedirs(Path(output_json_path).parent, exist_ok=True)
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n已輸出至：{output_json_path}")


自動問答中:   0%|          | 0/281 [00:00<?, ?it/s]


[Fallback] 使用 fallback 策略，因為 RAG 回答無效：<<UNANSWERABLE>>


自動問答中:   0%|          | 1/281 [01:05<5:06:01, 65.58s/it]


[Fallback] 使用 fallback 策略，因為 RAG 回答無效：<<UNANSWERABLE>>


自動問答中:   1%|          | 3/281 [02:01<2:39:55, 34.51s/it]

...


[Fallback] 使用 fallback 策略，因為 RAG 回答無效：<<UNANSWERABLE>>


自動問答中:  98%|█████████▊| 276/281 [1:17:49<02:39, 31.83s/it]


[Fallback] 使用 fallback 策略，因為 RAG 回答無效：<<UNANSWERABLE>>


自動問答中: 100%|██████████| 281/281 [1:18:36<00:00, 16.78s/it]


已輸出至：output/retrieval_topk.json


# evaluation

In [13]:
!git clone https://github.com/neulab/BARTScore.git
!pip install transformers
!pip install rouge-score
!pip install bert-score

Cloning into 'BARTScore'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194 (from 1)
Receiving objects: 100% (220/220), 101.98 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


In [15]:
import os
import json
import torch
from rouge_score import rouge_scorer
from bert_score import score
# 載入模組
import sys
sys.path.append("/content/BARTScore")
from bart_score import BARTScorer

EVAL_FILE = "output/retrieval_topk.json"

# === 初始化 BARTScorer ===
bart_scorer = BARTScorer(device='cuda:0' if torch.cuda.is_available() else 'cpu',
                          checkpoint='facebook/bart-large-cnn')

# === 字詞層級 F1 Score ===
def f1_score_fn(ref, pred):
    ref_tokens = set(ref.lower().split())
    pred_tokens = set(pred.lower().split())
    common = ref_tokens & pred_tokens
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return f1 * 100

# === ROUGE 分數 ===
def rouge_score_fn(ref, pred):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ref, pred)

    def extract(score_obj):
        return score_obj.precision * 100, score_obj.recall * 100, score_obj.fmeasure * 100

    r1_p, r1_r, r1_f = extract(scores['rouge1'])
    r2_p, r2_r, r2_f = extract(scores['rouge2'])
    rl_p, rl_r, rl_f = extract(scores['rougeL'])

    return (r1_p, r1_r, r1_f,
            r2_p, r2_r, r2_f,
            rl_p, rl_r, rl_f)

# === BERTScore ===
def bs_score(ref, pred):
    P, R, F1 = score([pred], [ref], lang="en", verbose=False)
    return P.mean().item() * 100, R.mean().item() * 100, F1.mean().item() * 100

# === BARTScore（對稱平均）===
def bart_score_wrapper(src, tgt):
    forward = bart_scorer.score([src], [tgt])[0]
    backward = bart_scorer.score([tgt], [src])[0]
    return ((forward + backward) / 2)

# === 主評估函式 ===
def evaluate(file_path, pred_key="answer", ref_key="reference",
             start_id=0, end_id=None, bs_true=True, filter_fallback=None):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    f1_total = bart_total = 0
    r1_p_total = r1_r_total = r1_f1_total = 0
    r2_p_total = r2_r_total = r2_f1_total = 0
    rl_p_total = rl_r_total = rl_f1_total = 0
    bs_p_total = bs_r_total = bs_f1_total = 0

    count = 0
    end_id = len(data) if end_id is None else min(end_id + 1, len(data))

    for i in range(start_id, end_id):
        item = data[i]
        if filter_fallback is not None and item.get("used_fallback") != filter_fallback:
            continue

        pred = item.get(pred_key, "").strip()
        ref = item.get(ref_key, "").strip()
        if not pred or not ref:
            continue

        f1_total += f1_score_fn(ref, pred)

        (r1p, r1r, r1f,
         r2p, r2r, r2f,
         rlp, rlr, rlf) = rouge_score_fn(ref, pred)

        r1_p_total += r1p
        r1_r_total += r1r
        r1_f1_total += r1f

        r2_p_total += r2p
        r2_r_total += r2r
        r2_f1_total += r2f

        rl_p_total += rlp
        rl_r_total += rlr
        rl_f1_total += rlf

        if bs_true:
            bsp, bsr, bsf = bs_score(ref, pred)
            bs_p_total += bsp
            bs_r_total += bsr
            bs_f1_total += bsf
            bart_total += bart_score_wrapper(ref, pred)

        count += 1

    fallback_label = {
        True: "使用 fallback",
        False: "未使用 fallback",
        None: "全部"
    }

    print(f"\n評估結果（{fallback_label[filter_fallback]}）共 {count} 筆：")
    if count == 0:
        print("沒有符合條件的資料。")
        return

    print(f"F1 Score:  {f1_total / count:.2f}")
    print(f"ROUGE-1:   P={r1_p_total/count:.2f}  R={r1_r_total/count:.2f}  F1={r1_f1_total/count:.2f}")
    print(f"ROUGE-2:   P={r2_p_total/count:.2f}  R={r2_r_total/count:.2f}  F1={r2_f1_total/count:.2f}")
    print(f"ROUGE-L:   P={rl_p_total/count:.2f}  R={rl_r_total/count:.2f}  F1={rl_f1_total/count:.2f}")
    if bs_true:
        print(f"BERTScore: P={bs_p_total/count:.2f}  R={bs_r_total/count:.2f}  F1={bs_f1_total/count:.2f}")
        print(f"BARTScore: {bart_total / count:.2f}")

# === 直接執行評估（針對固定檔案）===
if __name__ == "__main__":
    FILE = EVAL_FILE

    print("\n===== 不使用 fallback 的評估 =====")
    evaluate(file_path=FILE, filter_fallback=False)

    print("\n===== 使用 fallback 的評估 =====")
    evaluate(file_path=FILE, filter_fallback=True)

    print("\n===== 全部資料的總體評估 =====")
    evaluate(file_path=FILE, filter_fallback=None)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]


評估結果（未使用 fallback）共 148 筆：
F1 Score:  21.48
ROUGE-1:   P=32.31  R=27.74  F1=27.17
ROUGE-2:   P=8.72  R=7.12  F1=7.08
ROUGE-L:   P=22.26  R=18.87  F1=18.44
BERTScore: P=86.09  R=85.63  F1=85.85
BARTScore: -3.78

===== 使用 fallback 的評估 =====

評估結果（使用 fallback）共 133 筆：
F1 Score:  21.26
ROUGE-1:   P=22.23  R=43.39  F1=27.46
ROUGE-2:   P=5.21  R=10.64  F1=6.56
ROUGE-L:   P=13.84  R=27.54  F1=17.17
BERTScore: P=83.83  R=85.70  F1=84.75
BARTScore: -3.82

===== 全部資料的總體評估 =====

評估結果（全部）共 281 筆：
F1 Score:  21.37
ROUGE-1:   P=27.54  R=35.15  F1=27.31
ROUGE-2:   P=7.06  R=8.78  F1=6.83
ROUGE-L:   P=18.28  R=22.97  F1=17.84
BERTScore: P=85.02  R=85.66  F1=85.33
BARTScore: -3.80
